In [1]:
import pandas as pd
import numpy as np
import os, os.path
import sqlite3


In [2]:
Tags = pd.read_csv("Tags.csv.gz",
compression = 'gzip')
Votes = pd.read_csv("Votes.csv.gz",
compression = 'gzip')
Users = pd.read_csv("Users.csv.gz",
compression = 'gzip')
Posts = pd.read_csv("Posts.csv.gz",
compression = 'gzip')
PostLinks = pd.read_csv("PostLinks.csv.gz",
compression = 'gzip')
Comments = pd.read_csv("Comments.csv.gz",
compression = 'gzip')
Badges = pd.read_csv("Badges.csv.gz",
compression = 'gzip')

In [3]:
baza = os.path.join('przyklad.db')

In [4]:
if os.path.isfile(baza): # jesli baza już istneje...
    os.remove(baza) # ...usuniemy ja
# zeby miec zaczac z ,,czystą kartą''
conn = sqlite3.connect(baza) # połączenie do bazy danych
Badges.to_sql("Badges", conn) # importujemy ramkę danych do bazy danych
Comments.to_sql("Comments", conn)
PostLinks.to_sql("PostLinks", conn)
Posts.to_sql("Posts", conn)
Tags.to_sql("Tags", conn)
Users.to_sql("Users", conn)
Votes.to_sql("Votes", conn)


In [5]:
sql_1 = pd.read_sql_query("""
SELECT Count, TagName
FROM Tags
WHERE Count > 1000
ORDER BY Count DESC
""", conn)



In [6]:
#wczytanie 
pd_1 = Tags[["Count", "TagName"]]
#wyciągnięcie wartości Count >1000, sortownie, zresetowanie indexu
pd_1 = pd_1.loc[pd_1['Count'] > 1000].sort_values(by=['Count'], ascending = False).reset_index(drop = True)
#porównanie
pd_1.equals(sql_1)


True

In [7]:
sql_2 = pd.read_sql_query("""
SELECT Location, COUNT(*) AS Count
FROM (
SELECT Posts.OwnerUserId, Users.Id, Users.Location
FROM Users
JOIN Posts ON Users.Id = Posts.OwnerUserId
)
WHERE Location NOT IN ('')
GROUP BY Location
ORDER BY Count DESC
LIMIT 10
""", conn)
sql_2

,Location,Count
0,"Christchurch, New Zealand",2765
1,"New York, NY",1788
2,"London, United Kingdom",1708
3,UK,1590
4,"Sunshine Coast QLD, Australia",1550
5,Australia,1183
6,"Vancouver, Canada",967
7,Netherlands,935
8,on the server farm,924
9,Pennsylvania,921


In [8]:
#inner join
pd_2 = pd.merge(Users, Posts, left_on='Id', right_on='OwnerUserId', how = "inner")
#grupowanie
pd_2 = pd_2.groupby("Location").agg('Id_x').count().reset_index()
pd_2 = pd_2.rename({'Id_x': 'Count'}, axis=1) 
#sortowanie
pd_2 = pd_2.sort_values(by="Count", ascending=False).reset_index(drop=True)
pd_2 =pd_2.iloc[:10]

pd_2.equals(sql_2)

True

In [9]:
sql_3 = pd.read_sql_query("""
SELECT Year, SUM(Number) AS TotalNumber
FROM (
SELECT
Name,
COUNT(*) AS Number,
STRFTIME('%Y', Badges.Date) AS Year
FROM Badges
WHERE Class = 1
GROUP BY Name, Year
)
GROUP BY Year
ORDER BY TotalNumber
""", conn)
sql_3

,Year,TotalNumber
0,2011,16
1,2012,23
2,2013,66
3,2021,153
4,2014,197
5,2020,265
6,2015,328
7,2016,509
8,2017,552
9,2018,697


In [10]:
pd_3 = Badges
# tylko class = 1
pd_3 = pd_3.loc[ pd_3.Class == 1] 
# wyciągnięcie roku z daty
pd_3['Year'] = pd.DatetimeIndex(pd_3['Date']).year
#grupowanie i wyciąganie potrzebnych kolumn
pd_3 = pd_3.groupby(["Name", "Year"]).count().reset_index()
pd_3 = pd_3.rename({'Id': 'Number'}, axis=1)
pd_3 = pd_3.iloc[:, 0:3]
#ponowne grupownie
pd_3 = pd_3.groupby(["Year"])["Number"].apply(np.sum).reset_index().rename(columns={'Number': 'TotalNumber'})
#sortowanie
pd_3 = pd_3.sort_values(by="TotalNumber", ascending=True).reset_index(drop=True)

#zmiana kolumny year z object na int, co ma wedłóg mnie większy sens 
sql_3.Year = sql_3.Year.astype('int64')
pd_3.equals(sql_3)



<ipython-input-10-d01eecec4b05>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pd_3['Year'] = pd.DatetimeIndex(pd_3['Date']).year


True

In [11]:
sql_4 = pd.read_sql_query("""
SELECT
Users.AccountId,
Users.DisplayName,
Users.Location,
AVG(PostAuth.AnswersCount) as AverageAnswersCount
FROM
(
SELECT
AnsCount.AnswersCount,
Posts.Id,
Posts.OwnerUserId
FROM (
SELECT Posts.ParentId, COUNT(*) AS AnswersCount
FROM Posts
WHERE Posts.PostTypeId = 2
GROUP BY Posts.ParentId
) AS AnsCount
JOIN Posts ON Posts.Id = AnsCount.ParentId
) AS PostAuth
JOIN Users ON Users.AccountId=PostAuth.OwnerUserId
GROUP BY OwnerUserId
ORDER BY AverageAnswersCount DESC, AccountId ASC
LIMIT 10
""", conn)
sql_4

,AccountId,DisplayName,Location,AverageAnswersCount
0,280.0,csmba,"San Francisco, CA",11.0
1,40811.0,vocaro,"San Jose, CA",11.0
2,204.0,Josh,Australia,10.0
3,44093.0,Emma Arbogast,"Salem, OR",10.0
4,11758.0,rvarcher,"Oklahoma City, OK",9.0
5,19588.0,JD Isaacks,"Atlanta, GA",8.0
6,20473.0,Jeremy Boyd,"Houston, TX",8.0
7,42364.0,Petrogad,None,8.0
8,54571.0,Christian,None,8.0
9,79346.0,Thomas Matthews,California,8.0


In [12]:

pd_4 = Posts
# tylko PostTypeId = 2
pd_4 = pd_4[pd_4.PostTypeId == 2] 
#grupowanie
pd_4 = pd_4.groupby(["ParentId"]).count().reset_index().rename(columns={'Id': 'AnswersCount'}).iloc[:, 0:2]
#inner join
pd_4 = pd.merge(Posts, pd_4, left_on='Id', right_on='ParentId', how = "inner")
#potrzebne kolumny
pd_4 = pd_4[["AnswersCount", "Id", "OwnerUserId"]]
# obliczanie average count
pd_4 = pd_4.groupby(["OwnerUserId"])["AnswersCount"].agg(np.mean).reset_index().rename(columns={'AnswersCount': 'AverageAnswersCount'})
#inner join z Users
pd_4 = pd.merge(Users, pd_4, left_on='AccountId', right_on='OwnerUserId', how = "inner")
pd_4 = pd_4[["AccountId", "DisplayName", "Location", "AverageAnswersCount"]]
#sortwanie wobec 2 kolumn, ponieważ dla tych samych wartości AverageAnswersCount było inne sortwanie niż w sql
pd_4 = pd_4.sort_values(by=["AverageAnswersCount", "AccountId"], ascending=[False, True]).reset_index(drop=True).iloc[0:10]
pd_4 = pd_4.iloc[:10]

pd_4.equals(sql_4)

#planes[["engine"]].groupby("engine").size().reset_index().rename(columns={0: 'count'})

True

In [13]:
sql_5 = pd.read_sql_query("""
SELECT Posts.Title, Posts.Id,
STRFTIME('%Y-%m-%d', Posts.CreationDate) AS Date,
VotesByAge.Votes
FROM Posts
JOIN (
SELECT
PostId,
MAX(CASE WHEN VoteDate = 'new' THEN Total ELSE 0 END) NewVotes,
MAX(CASE WHEN VoteDate = 'old' THEN Total ELSE 0 END) OldVotes,
SUM(Total) AS Votes
FROM (
SELECT
PostId,
CASE STRFTIME('%Y', CreationDate)
WHEN '2021' THEN 'new'
WHEN '2020' THEN 'new'
ELSE 'old'
END VoteDate,
COUNT(*) AS Total
FROM Votes
WHERE VoteTypeId IN (1, 2, 5)
GROUP BY PostId, VoteDate
) AS VotesDates
GROUP BY VotesDates.PostId
HAVING NewVotes > OldVotes
) AS VotesByAge ON Posts.Id = VotesByAge.PostId
WHERE Title NOT IN ('')
ORDER BY Votes DESC
LIMIT 10
""", conn)
sql_5

,Title,Id,Date,Votes
0,What's the longest distance that can be travel...,151994,2020-01-09,140
1,Considerations for very fragile and expensive ...,157138,2020-05-10,108
2,What are these chair-like things in hotels?,153605,2020-02-10,94
3,What to do if I overstayed my e-visa for Saint...,152418,2020-01-18,84
4,What are the hidden fees of a cruise?,152178,2020-01-13,80
5,What happens when a town is under quarantine a...,152677,2020-01-23,76
6,I've been warned to leave the US within 10 day...,156945,2020-04-28,72
7,"When an individual enters the United States, c...",159298,2020-09-06,68
8,Is Seiryu Miharashi Station the only train sta...,162386,2021-02-04,64
9,Positive drug test in Singapore: How long do I...,163264,2021-04-08,64


In [14]:
pd_5 = Votes
#VoteTypeId = 1, 2 lub 5
pd_5 = pd_5[pd_5.VoteTypeId.isin([1,2,5])]
#wyciągnięcie roku z daty
pd_5['Year'] = pd.DatetimeIndex(pd_5['CreationDate']).year
#stworzenie nowych kolumn zliczających old i new votes
pd_5['VoteDate'] = np.where(pd_5['Year'] >2019 , 'new', 'old')
pd_5 = pd_5.groupby(["PostId", "VoteDate"]).count().reset_index().rename(columns={'Id': 'Total'}).iloc[:,0:3]
pd_5['OldVotes'] = np.where(pd_5['VoteDate'] == 'old' , pd_5['Total'], 0)
pd_5['NewVotes'] = np.where(pd_5['VoteDate'] == 'new' , pd_5['Total'], 0)
# grupowanie by zsumować łączne głosy danego użytkownika
pd_5 = pd_5.groupby(["PostId"])["Total", "OldVotes", "NewVotes"].agg(np.sum).reset_index().rename(columns={'Total': 'Votes'})
#NewVotes > OldVotes
pd_5 = pd_5[pd_5.NewVotes > pd_5.OldVotes]
#inner join z Posts
pd_5 = pd.merge(pd_5, Posts, left_on='PostId', right_on='Id', how = "inner")
#usunięcie pustych tytułów
pd_5 = pd_5[~pd_5.Title.isnull()]
#wyciągnięcie daty
pd_5['Date'] = [x[:10] for x in pd_5['CreationDate']]
pd_5 = pd_5[["Title", "Id","Date" , "Votes"]]
#sortwanie, znowy trzeba wedłóg 2 kolumn
pd_5 = pd_5.sort_values(by=["Votes","Title"], ascending=[False, True]).reset_index(drop=True).iloc[0:10]

pd_5.equals(sql_5)


<ipython-input-14-8eafb586c153>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pd_5['Year'] = pd.DatetimeIndex(pd_5['CreationDate']).year
<ipython-input-14-8eafb586c153>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pd_5['VoteDate'] = np.where(pd_5['Year'] >2019 , 'new', 'old')
<ipython-input-14-8eafb586c153>:12: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  pd_5 = pd_5.groupby(["PostId"])["Total", "OldVotes", "NewVotes"].

True